In [86]:
import sympy as sym
from sympy import S
import math
sym.init_printing()

<div>
<img src="../q8bot_documentation/FK_diagram_1.jpg" width="360"/>
</div>

Let's look at the parallel five-bar linkage up-side down from the robot for a better view.
Since q1, q2, l1, l2, l1', l2', and d are known, we can represent point A and point B's coordinates as follow:

$x_A = d + l_1cos(q_1)$ &nbsp;&nbsp;&nbsp; (1)

$y_A = l_1\cdot sin(q_1)$ &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; (2)

$x_B = l_1^{'}\cdot cos(q_2)$ &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; (3)

$y_B = l_1^{'}\cdot sin(q_2)$ &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; (4)

In [87]:
X, Y, Xa, Ya, Xb, Yb, q1, q2, l1, l2, l1p, l2p, d = sym.symbols('X, Y, Xa, Ya, Xb, Yb, q1, q2, l1, l2, l1p, l2p, d', real=True)

Eq1 = sym.Eq(d+l1*sym.cos(q1), Xa)
Eq2 = sym.Eq(  l1*sym.sin(q1), Ya)
Eq3 = sym.Eq( l1p*sym.cos(q2), Xb)
Eq4 = sym.Eq( l1p*sym.sin(q2), Yb)

sym.solve([Eq1,Eq2,Eq3,Eq4], (Xa,Ya,Xb,Yb))

{Xa: d + l₁⋅cos(q₁), Xb: l1p⋅cos(q₂), Ya: l₁⋅sin(q₁), Yb: l1p⋅sin(q₂)}

<div>
<img src="../q8bot_documentation/FK_diagram_2.jpg" width="540"/>
</div>

To find the coordinate of point C, we can think of it as one of the intersections of two circles in the xy coordinate system. The two circles can be written as:

$l_2^2 = (x - x_A)^2 + (y - y_A)^2$    &nbsp;&nbsp;&nbsp;&nbsp; (5)

$l_2^{'2} = (x - x_B)^2 + (y - y_B)^2$ &nbsp;&nbsp;&nbsp; (6)

In [89]:
Eq5 = sym.Eq( l2**2, (X - Xa)**2+(Y - Ya)**2)
Eq6 = sym.Eq(l2p**2, (X - Xb)**2+(Y - Yb)**2)

sym.solve([Eq5,Eq6], (X,Y))

⎡⎛ ⎛                                                     ⎛     _______________
⎢⎜ ⎜                                                     ⎜    ╱  ⎛    2       
⎢⎜ ⎜    2     2     2     2     2      2                 ⎜  ╲╱  -⎝- Xa  + 2⋅Xa
⎢⎜-⎜- Xa  + Xb  - Ya  + Yb  + l₂  - l2p  + (2⋅Ya - 2⋅Yb)⋅⎜- ──────────────────
⎢⎜ ⎜                                                     ⎜                    
⎢⎜ ⎝                                                     ⎝                    
⎢⎜────────────────────────────────────────────────────────────────────────────
⎢⎜                                                                            
⎣⎝                                                                            

______________________________________________________________________________
        2     2               2     2                 2⎞ ⎛    2               
⋅Xb - Xb  - Ya  + 2⋅Ya⋅Yb - Yb  + l₂  - 2⋅l₂⋅l2p + l2p ⎠⋅⎝- Xa  + 2⋅Xa⋅Xb - Xb
───────────────────────────────────────────────────

This pure algebraic solution looks overly complicated. Luckily, it becomes much more manageable once we plug in all known values.

In [92]:
# Input known values here
q1  = 45
q2  = 120
d   = 20
l1  = 25
l1p = 25
l2  = 35
l2p = 35

# Convert deg to rad
q1 = q1 * math.pi / 180
q2 = q2 * math.pi / 180

Eq1 = sym.Eq(d+l1*sym.cos(q1), Xa)
Eq2 = sym.Eq(  l1*sym.sin(q1), Ya)
Eq3 = sym.Eq( l1p*sym.cos(q2), Xb)
Eq4 = sym.Eq( l1p*sym.sin(q2), Yb)
soln_1 = sym.solve([Eq1,Eq2,Eq3,Eq4], (Xa,Ya,Xb,Yb))

Xa, Ya, Xb, Yb = round(soln_1[Xa],2), round(soln_1[Ya],2), round(soln_1[Xb],2), round(soln_1[Yb],2)
print(Xa, Ya, Xb, Yb)

37.68 17.68 -12.50 21.65


In [94]:
Eq7 = sym.Eq( l2**2, (X - Xa)**2 + (Y - Ya)**2)
Eq8 = sym.Eq(l2p**2, (X - Xb)**2 + (Y - Yb)**2)

soln_2 = sym.solve([Eq7,Eq8], (X,Y))
print(soln_2)

[(10.6733246744563, -4.58026509751920), (14.5087365749702, 43.9104030658175)]


This takes about 10 seconds. Not ideal. Let's try to optimize. By using scipy instead of sympy, we can solve the same equation much faster.

In [18]:
import math
import numpy as np
from scipy.optimize import fsolve

# Input known values here
q1  = 57.2
q2  = 131.1
d   = 19.5
l1  = 25
l1p = 25
l2  = 35
l2p = 35

# Convert deg to rad
q1 = q1 * math.pi / 180
q2 = q2 * math.pi / 180

# Using scipy's fsolve, which is known to be faster at this type of calculation
def func(x, *angle):
    Q1, Q2 = angle
    Xa = l1 * math.cos(Q1) + d
    Ya = l1 * math.sin(Q1)
    Xb = l1p * math.cos(Q2)
    Yb = l1p * math.sin(Q2)
    return [x[0]*x[0] - 2*x[0]*Xa + Xa**2 + x[1]*x[1] - 2*x[1]*Ya + Ya**2 - l2*l2,
            x[0]*x[0] - 2*x[0]*Xb + Xb**2 + x[1]*x[1] - 2*x[1]*Yb + Yb**2 - l2p*l2p]

angle = (q1, q2)
x, y = fsolve(func, [10, 60], args=angle)
print(x, y)

7.217825557905917 44.63778861393392


This is great! However, keep in mind that if we ever need to run this with cpp on a microcontroller, we won't be able to use these fancy libraries. Therefore, it will be best if we can represent the coordinate of point C in an agregated expression.

http://ambrnet.com/TrigoCalc/Circles2/circle2intersection/CircleCircleIntersection.htm